In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import json as js

In [2]:
df = pd.read_csv('data/resume.csv')

In [3]:
df.shape

(2369, 20)

In [4]:
df.duplicated().shape

(2369,)

In [5]:
df.head(1)

,Unnamed: 0,age,auto,business_trips,categories,citizenship,city,education,employment,experience,experience_length,family,gender,languages,metro_or_moving,name,photo_url,salary,skills,url
0,0,20,NaN,не готов к командировкам,Архитектура,Россия,Ковров,"[{'kind': 'Неполное высшее образование', 'form...",неполный рабочий день,"[{'period': 'сентябрь 2017 – март 2018', 'leng...",7 месяцев,NaN,f,"['Немецкий язык, Базовый']",NaN,"3D-моделлер, IT стажёр, консультант",NaN,30 000₽,NaN,https://www.superjob.ru/resume/3d-modeller-436...


In [6]:
df.columns

Index(['Unnamed: 0', 'age', 'auto', 'business_trips', 'categories',
       'citizenship', 'city', 'education', 'employment', 'experience',
       'experience_length', 'family', 'gender', 'languages', 'metro_or_moving',
       'name', 'photo_url', 'salary', 'skills', 'url'],
      dtype='object')

In [7]:
df = df.drop('Unnamed: 0', axis=1)

In [8]:
df.auto.value_counts(dropna=False)

['B — легковые авто']                                                                                 1171
NaN                                                                                                    808
['B — легковые авто', 'C — грузовые авто']                                                             220
['A — мотоциклы', 'B — легковые авто']                                                                  74
['A — мотоциклы', 'B — легковые авто', 'C — грузовые авто']                                             52
['B — легковые авто', 'C — грузовые авто', 'D — автобусы']                                              11
['C — грузовые авто']                                                                                   11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы', 'E — авто с прицепом']                        7
['B — легковые авто', 'D — автобусы']                                                                    5
['A — мотоциклы', 'B — легковые авто'

In [9]:
df.auto=df.auto.fillna("['Нет прав']")

## Замена - не выкидываем

In [10]:
df.business_trips.value_counts(dropna=False)

NaN                         1373
не готов к командировкам     968
на дому                       28
Name: business_trips, dtype: int64

In [11]:
df.url.iloc[1]

'https://www.superjob.ru/resume/administrator-26936498.html'

## Замена, ничего не понятно, почему либо не готов, либо на дому, где вариант готов?

In [12]:
#df = df.drop('business_trips', axis=1)

In [13]:
df.categories.value_counts(dropna=False)

Системное администрирование                      552
NaN                                              467
Техническая поддержка                            358
IT, Интернет, связь, телеком                     238
Инжиниринг                                       184
Администрирование баз данных                      80
Административная работа, секретариат, АХО         64
Сетевые технологии                                49
Разработка, программирование                      41
Передача данных и доступ в интернет               38
Управление проектами                              34
Аналитика                                         31
Интернет, создание и поддержка сайтов             23
Другое                                            18
Электронный документооборот                       15
Внедрение и сопровождение ПО                      12
Делопроизводство, ввод данных, систематизация     11
Компьютеры, оргтехника, ПО                         9
Web-верстка                                   

In [14]:
df.categories=df.categories.fillna('Не указано')

Выкинуть незаполненные (20 % данных) или весь признак? Неясно
## Выкинуть всегда успеем, преобразовал в отдельную категорию.

Так как категории заполнялись сайтом автоматически, эту информацию можно извлечь из других полей (name), удаляем.
## Хотя может ты и прав, короче потом подумаем.

In [15]:
#df.cit

In [16]:
df.citizenship.value_counts(dropna=False)

Россия               1615
NaN                   727
Украина                 8
Казахстан               7
Кыргызстан              3
Азербайджан             3
Беларусь                2
Молдова                 1
Дальнее зарубежье       1
Литва                   1
Таджикистан             1
Name: citizenship, dtype: int64

## Заменим

Считаем, что гражданство не влияет на уровень зп.

In [17]:
df.citizenship=df.citizenship.fillna('Да вообще плевать')

In [18]:
ctgr = pd.get_dummies(df.citizenship)
cit = ctgr['Россия']
ctgr = ctgr.drop(['Россия'], axis=1)
city_oth = np.sum(ctgr.values, axis=1)
df['is_cit_p'] = cit
df['isn_t_cit_p'] = city_oth
df = df.drop(['citizenship'], axis=1)
df.shape

(2369, 20)

In [19]:
df.city.value_counts(dropna=False)

Москва                               720
Санкт-Петербург                      293
Хабаровск                             64
Уфа                                   55
Иркутск                               51
Ростов-на-Дону                        48
Краснодар                             33
Самара                                32
Казань                                27
Саратов                               27
Рязань                                26
Нижний Новгород                       24
Пенза                                 23
Ульяновск                             23
Волгоград                             22
Воронеж                               20
Оренбург                              19
Тверь                                 17
Сыктывкар                             17
Челябинск                             16
Владимир                              16
Смоленск                              16
Екатеринбург                          16
Астрахань                             15
Тюмень          

Обучать модель для регионов отдельно не хочется, будет слишком мало объектов. Предложение: выделить категории в зависимости от уровня зарплат по региону. Для начала 3: Мск, Спб, остальные.

In [20]:
ctgr = pd.get_dummies(df.city)
city_Msk = ctgr['Москва']
city_Spb = ctgr['Санкт-Петербург']
ctgr = ctgr.drop(['Москва', 'Санкт-Петербург'], axis=1)
city_oth = np.sum(ctgr.values, axis=1)
df['city_Msk'] = city_Msk
df['city_Spb'] = city_Spb
df['city_oth'] = city_oth
df = df.drop(['city'], axis=1)
df.shape

(2369, 22)

In [21]:
df.employment.value_counts(dropna=False)

полная занятость           1180
NaN                        1107
неполный рабочий день        33
сменный график работы        25
частичная занятость          13
работа вахтовым методом      11
Name: employment, dtype: int64

In [22]:
ctgr = pd.get_dummies(df.employment)
full_empl = ctgr['полная занятость'] | ctgr['сменный график работы']
not_full = ctgr.drop(['полная занятость', 'сменный график работы'], axis=1)
nf = np.sum(not_full.values, axis=1)
df['ready_for_full']=full_empl
df['not_ready_for_full']=nf
df=df.drop(['employment'], axis=1)
df.shape

(2369, 23)

Много NaN, выкидываем.
## Очень важная же инфа

In [25]:
df.family.value_counts(dropna=False)

cостоит в браке, есть дети       804
NaN                              687
не состоит в браке, детей нет    528
cостоит в браке, детей нет       168
не состоит в браке, есть дети    111
cостоит в браке                   37
не состоит в браке                29
есть дети                          3
детей нет                          2
Name: family, dtype: int64

In [26]:
df = df.drop('family', axis=1)

In [27]:
gender = pd.get_dummies(df.gender)
gender.f
df['gender'] = gender.f

Как вариант можно считать, что если иностранные языки не указаны, то человек ими не владеет. Логика в этом есть!!!

In [28]:
df.languages.value_counts(dropna=False)

['Английский язык, Технический']                                                                                      832
NaN                                                                                                                   547
['Английский язык, Базовый']                                                                                          463
['Английский язык, Разговорный']                                                                                      164
['Английский язык, Технический', 'Немецкий язык, Базовый']                                                             47
['Английский язык, Базовый', 'Немецкий язык, Базовый']                                                                 42
['Английский язык, Cвободное владение']                                                                                40
['Английский язык, Технический', 'Русский язык, Cвободное владение']                                                   16
['Английский язык, Техни

In [29]:
#df = df.drop('languages', axis=1)
df.languages=df.languages.fillna("[]")

In [65]:
def i_am_cruel(x):
    #print (eval(x))
    for z in eval(x):
        y = z.split(', ')
        if y[0] == 'Английский язык':
            if y[1] == 'Базовый':
                return 1
            if y[1] == 'Технический':
                return 2
            if y[1] == 'Разговорный':
                return 3
            if y[1] == 'Cвободное владение':
                return 4
            print (y[1])
    return 0
for i in df.languages.values:
    i_am_cruel(i)
df['English_lvl']=df.languages.apply(i_am_cruel)

In [68]:
df.drop('languages', axis=1, inplace=True)

In [ ]:
Lang = set()
for x in df.languages.values:
    if type(x)==float:
        Lang.add('С')
    print (x)
    for y in eval(x):
        Lang.add(y)
Lang

In [ ]:
df.metro_or_moving.value_counts(dropna=False)

## Не сделано, но может быть лучше рабить на готов-не готов к переезду?

In [ ]:
#df = df.drop('metro_or_moving', axis=1)

In [ ]:
df.name.value_counts(dropna=False)

In [ ]:
df.name.unique().shape

Нужно обрабатывать как текст. Очень много синонимов!

In [ ]:
df = df.drop('photo_url', axis=1)

In [ ]:
df.skills.value_counts(dropna=False).shape

Часть - дубликаты. Но полных дубликатов не было. Видимо, люди хотят на разные должности по одинаковым резюме. Можно оставить как есть (логике задачи это не противоречит)

In [ ]:
df.groupby(by='skills')

In [ ]:
df[df.skills.isna()].shape

In [ ]:
df = df.dropna(subset=['skills'])

## Я плох в панде, что здесь происходит?

Видимо, придется выкинуть незаполненные, важная инфа :(

In [ ]:
df = df.drop('url', axis = 1)

In [ ]:
df.shape

In [ ]:
df.education.value_counts(dropna=False)

## Тяжело
нашлись уникумумы с 4 образованиями

In [ ]:
A=set()

In [ ]:
A=set()
for x in df.education.values:
    if type(x)==float:
        A.add("Пусто")
    else:
        for ed in eval(x):
            if ed['kind']=='Учащийся школы':
                print(x)
            A.add(ed['kind'])

In [ ]:
A

In [ ]:
df.education.value_counts(dropna=False)

Просто интересно.

def returner_because_i_am_not_clever(x):
    if type(x)==float:
        return "пусто"
    if len(eval(x))==4:
        #print (x)
        print ("^^^^^^^^^^
    
    return eval()
set([returner_because_i_am_not_clever(x) for x in df.education.values])

In [ ]:
#df = df[df.education.isna() == False]
df = df.dropna(subset=['education'])

Выкидываем незаполненные. Че дальше - непонятно

In [ ]:
df.experience_length.value_counts(dropna=False)

Наны, видимо, - нет опыта работы

In [ ]:
df.experience_length=df.experience_length.fillna("0 лет и 0 месяцев")

In [ ]:
#df = df.dropna(subset=['experience_length'])

In [ ]:
df.shape

In [ ]:
df.index = np.arange(df.shape[0])

In [ ]:
df.columns

### Промежуточный итог: ###

Остались пропуски в 'languages', нужно решить, что с ними делать. Идея в том, что NaN можно интерпретировать как незнание языков. 

В остальных фичах пропусков нет. 

Город и пол one-hot.

Что с образованием - неясно, очень много категорий.

Для 'experience' и 'skills' нужно посмотреть распределения N-грамм и выбрать хорошие, информативные фичи по вхождениям ключевых слов.

Так же необходимо все числовые признаки нормализовать.

In [ ]:
def period_to_month(string):
    s = string.split()
    if len(s) == 2:
        if (s[1] == 'год') or (s[1] == 'года') or (s[1] == 'лет'):
            return int(s[0]) * 12
        return int(s[0])
    return int(s[0]) * 12 + int(s[3])

In [ ]:
pers = df.experience_length.copy()

In [ ]:
pers is df.experience_length

In [ ]:
for i in pers.index:
    pers.values[i] = period_to_month(pers.values[i])

In [ ]:
df.experience_length = pers
df.experience_length

Перевели стаж работы в месяцы.

add more text

In [ ]:
for x in df.salary.values:
    if (x[-1]!='₽'):
        print (x)

ЗП только в рублях, что очень странно

In [ ]:
df.salary=df.salary.apply(lambda x: int(x[:-1].replace(" ", '')))

In [ ]:
df.head(3)

In [ ]:
df.salary.value_counts